In [33]:
import matplotlib.pyplot as plt
import numpy as np
import astropy.units as u
from datetime import datetime
import pickle, json, sys, os, glob
import pandas as pd
from scipy.optimize import curve_fit
from scipy.stats import chi2
from scipy import optimize
import subprocess

from astropy.coordinates import SkyCoord
from lstchain.io.config  import get_standard_config
from ctapipe.io          import read_table
import tables

# Other auxiliar scripts
sys.path.insert(0, os.getcwd() + "/../scripts/")
import auxiliar as aux
import geometry as geom
import lstpipeline

import logging
logger = logging.getLogger(__name__)
logger.addHandler(logging.StreamHandler())
logger.setLevel(logging.INFO)

In [34]:
""" Source name in order to just complete the results file, and
in order to improve run organization."""
source_name = "crab"

# Standard paths for data in the IT cluster ---------
root_dl1 = "/fefs/aswg/data/real/DL1/*/v0.*/tailcut84/"
root_rfs = "/fefs/aswg/data/models/AllSky/20230927_v0.10.4_crab_tuned/"
root_mcs = "/fefs/aswg/data/mc/DL2/AllSky/20230927_v0.10.4_crab_tuned/TestingDataset/"

# Root path of this script
root = os.getcwd() + "/"
# Path to store the configuration file we are going to use
config_file = root + "config/standard_config.json"
# Path to store objects
root_objects = root + f"objects/"
# Data main directory
root_data = root + f"../../data/cherenkov_transparency_corrections/{source_name}/"

# Directories for the data
dir_dl1b_scaled = root_data + "dl1_scaled/"
dir_dl1m_scaled = root_data + "dl1_merged_scaled/"
dir_dl2_scaled  = root_data + "dl2_scaled/"
dir_dl2         = root_data + "dl2/"
dir_dl3_scaled_base = root_data + "dl3_scaled/"
dir_dl3_base    = root_data + "dl3/"
dir_irfs        = root_data + "irfs/"

In [35]:
runs_dataset_0 = [
   2853, 2922, 2923, 2924, 2934, 2950, 2959, 2960, 2965, 2967, 2968,
   2969, 2971, 2972, 2974, 2975, 2977, 2988, 2989, 2990, 2991, 2992,
   3009, 3011, 3012, 3087, 3088, 3089, 3093, 3095, 3096, 3097, 3098,
   3100, 3144, 3169, 3170, 3231, 3232, 3235, 3236, 3237, 3243, 3244,
   3245, 3265, 3266, 3267, 3268, 3269, 3270, 3271, 3272, 3273, 3274,
   3275, 3276, 3277, 3278, 3318, 3319, 3320, 3321, 3328, 3329, 3338,
   3339, 3340, 3355, 3356, 3382, 3515, 3516, 3583, 3584, 3585, 3586,
   3598, 3599, 3600, 3601, 3631, 3632, 3633, 3634, 3635, 3636, 3637,
   3638, 3639, 3650, 3651, 3652, 3653, 3655, 3672, 3673, 3674, 3675,
   3676, 3677, 3678, 3679, 3680, 3683, 3684, 3703, 3704, 3705, 3706,
   3707, 3708, 3709, 3710, 3711, 3712, 3725, 3730, 3894, 3895, 3953,
   3956, 3957, 3959, 3961, 3962, 3983, 4008, 4009, 4010, 4011, 4012,
   4013, 4014, 4015, 4033, 4034, 4035, 4036, 4067, 4068, 4069, 4073,
   4074, 4075, 4086, 4087, 4088, 4089, 4090, 4094, 4125, 4126, 4127,
   4128, 4129, 4145, 4146, 4147, 4148, 4149, 7084, 7087, 7088, 7097,
   7099, 7100, 7101, 7102, 7106, 7133, 7134, 7135, 7136, 7137, 7140,
   7141, 7142, 7143, 7168, 7169, 7170, 7172, 7173, 7174, 7196, 7197,
   7201, 7202, 7227, 7228, 7230, 7231, 7232, 7233, 7234, 7235, 7236,
   7253, 7254, 7275, 7276, 7277, 7278, 7279, 7282, 7301, 7302, 7304
]

runs_dataset_1 = [
    2914, 2929, 2930, 2931, 2932, 2933, 2934, 2949, 2950, 2967, 2968,
    2969, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2977, 2988, 2989,
    2990, 2991, 2992, 3004, 3005, 3006, 3007, 3008, 3093, 3094, 3095,
    3096, 3231, 3232, 3243, 3270, 3271, 3272, 3273, 3274, 3275, 3276,
    3277, 3278, 3279, 3318, 3319, 3320, 3321, 3328, 3329, 3330, 3338,
    3339, 3340, 3355, 3356, 3373, 3598, 3599, 3600, 3601, 3615, 3632,
    3633, 3634, 3635, 3672, 3673, 3674, 3675, 3676, 3677, 3706, 3707,
    3708, 4067, 4068, 4086, 4087, 6045, 6073, 6304, 6872, 6873, 6874,
    6875, 6892, 6893, 6894, 6895, 7097, 7098, 7099, 7133, 7136, 7161,
    7195, 7196, 7197, 7199, 7200, 7227, 7228, 7231, 7232, 7233, 7253,
    7254, 7255, 7256, 7274, 7275, 7276, 7277
]

runs_dataset_2 = [
    2914, 2929, 2930, 2931, 2932, 2933, 2949, 2950, 
    3004, 3005, 3006, 3007, 3008, 3093, 3279, 3338, 
    3355, 3598, 3600, 3632, 3633, 3672, 4067, 4068, 
    7161, 7195, 7199, 7200, 7256, 7274]

runs_dataset_3 = [
    2969, 2977, 2992, 3094, 3231, 3243, 3272,
    3599, 3676, 3706, 4086, 6045, 6875, 6893,
    6894, 7097, 7099, 7254, 7255, 7275, 7277]

runs_dataset_4 = [
    2934, 2967, 2968, 2970, 2971, 2972, 2973, 2974, 2975, 2976, 2988,
     2989, 2990, 2991, 3095, 3096, 3232, 3270, 3271, 3273, 3274, 3275,
     3276, 3277, 3278, 3318, 3319, 3320, 3321, 3328, 3329, 3330, 3339,
     3340, 3356, 3373, 3601, 3615, 3634, 3635, 3673, 3674, 3675, 3677,
     3707, 3708, 4087, 6073, 6304, 6872, 6873, 6874, 6892, 6895, 7098,
     7133, 7136, 7196, 7197, 7227, 7228, 7231, 7232, 7233, 7253, 7276]




run_numbers = [runs_dataset_0, runs_dataset_1, runs_dataset_2, runs_dataset_3, runs_dataset_4]
names = ["0", "1", "2", "3", "4"]

In [36]:
dl3_runs_individual_scaled = np.sort(glob.glob(dir_dl3_scaled_base + "*/*.fits"))
dl3_runs_individual = np.sort(glob.glob(dir_dl3_base + "*/*.fits"))

dl3_run_numbers_scaled = [int(f.split("Run")[-1].split(".")[0]) for f in dl3_runs_individual_scaled]
dl3_run_numbers = [int(f.split("Run")[-1].split(".")[0]) for f in dl3_runs_individual]

for j in range(len(run_numbers)):

    _run_numbers = run_numbers[j]
    _name = names[j]

    print(f"Moving {len(_run_numbers)}...\nFirst of all the scaled runs...")

    mv_folder = f"{dir_dl3_base}../dl3_{_name}/"
    mv_folder_scaled = f"{dir_dl3_scaled_base}../dl3_scaled_{_name}/"
    for path in [mv_folder, mv_folder_scaled]:
        os.makedirs(os.path.join(path), exist_ok=True)
    
    for i in range(len(dl3_runs_individual_scaled)):

        if dl3_run_numbers_scaled[i] in _run_numbers:

            command = f"cp {dl3_runs_individual_scaled[i]} {mv_folder_scaled}"
            subprocess.run(command, shell=True)

    print(f"Then the original ones...")
    
    for i in range(len(dl3_runs_individual)):

        if dl3_run_numbers[i] in _run_numbers:

            command = f"cp {dl3_runs_individual[i]} {mv_folder}"
            subprocess.run(command, shell=True)

Moving 208...
First of all the scaled runs...
Then the original ones...
Moving 117...
First of all the scaled runs...
Then the original ones...
Moving 30...
First of all the scaled runs...
Then the original ones...
Moving 21...
First of all the scaled runs...
Then the original ones...
Moving 66...
First of all the scaled runs...
Then the original ones...


### Add DL3 index files

In [37]:
for j in range(len(run_numbers)):

    _name = names[j]
    mv_folder = f"{dir_dl3_base}../dl3_{_name}/"
    mv_folder_scaled = f"{dir_dl3_scaled_base}../dl3_scaled_{_name}/"
    
    command = f"lstchain_create_dl3_index_files --input-dl3-dir {mv_folder} --overwrite"
    logger.info(command)
    subprocess.run(command, shell=True)
    
    command = f"lstchain_create_dl3_index_files --input-dl3-dir {mv_folder_scaled} --overwrite"
    logger.info(command)
    subprocess.run(command, shell=True)

lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/../dl3_0/ --overwrite
lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/../dl3_0/ --overwrite
lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/../dl3_0/ --overwrite
lstchain_create_dl3_index_files --input-dl3-dir /fefs/aswg/workspace/juan.jimenez/cherenkov_transparency_corrections/data_processing_srunwise/../../data/cherenkov_transparency_corrections/crab/dl3/../dl3_0/ --overwrite
2024-05-29 05:13:20,524 WARNING [lstchain.FITSIndexWriter] (lstchain_create_dl3_index_files.setup): Overwriting /fefs/aswg/w